In [16]:
import pymysql
import re
import pandas as pd
import numpy as np

food_db = pymysql.connect(
    user='1zo',
    passwd='multi1234!',
    host='13.112.232.65',
    db='tweets',
    charset='utf8mb4'
    
)
cursor = food_db.cursor(pymysql.cursors.DictCursor)

sql = "SELECT DISTINCT * FROM tweet_food"
cursor.execute(sql)
food_db = cursor.fetchall()

In [17]:
food_db[0]['text']

'#트친과_입맛궁합_알아보기\n5 맨날땡김\n4 거의땡김\n3 종종땡김\n2내돈주고안먹기\n1사줘도 잘안먹기\n\n육회: 3\n마라탕: 2\n간장게장: 1\n선지: 2\n민초: 3\n탄산수: 3\n커피: 5\n생굴: 1\n닭발: 3\n막창: 5\n곱창: 5\n닭똥집: 4\n알탕: 2\n산낙지: 4\n오뎅: 4\n양꼬치: 5'

In [18]:
len(food_db)

4748

In [19]:
col = ["id","육회","마라탕","간장게장","선지","민초","탄산수","커피","생굴","닭발","막창","곱창","닭똥집","알탕","산낙지","오뎅","양꼬치","떡볶이"]

df_food = pd.DataFrame(columns = col)
df_food

,id,육회,마라탕,간장게장,선지,민초,탄산수,커피,생굴,닭발,막창,곱창,닭똥집,알탕,산낙지,오뎅,양꼬치,떡볶이


In [20]:
#음식 전처리
for food in food_db:
    temp = {}
    food['text'] = food['text'].replace('\n','')
    food['username'] = food['username'].replace(' ','')
    for i in range(0,len(col)-1):
        if i == 0:
            temp[col[i]] = food['username']
        elif i != 0:
            data = re.search(col[i], food['text'])
            if data :
                temp_text = food['text'][data.span()[1]+2:data.span()[1]+7]
                numbers = re.findall("\d+", temp_text)
                if len(numbers) > 0 :
                    if int(numbers[0]) in range(0,6):
                        temp[col[i]] = numbers[0]
                    elif int(numbers[0]) > 5:
                        temp[col[i]] = 5
                else:
                    temp[col[i]] = -1
    df_food = df_food.append(temp, ignore_index=True)

In [21]:
df_food = df_food.replace(-1,np.NaN)
df_food

,id,육회,마라탕,간장게장,선지,민초,탄산수,커피,생굴,닭발,막창,곱창,닭똥집,알탕,산낙지,오뎅,양꼬치,떡볶이
0,Adorable_Lana,3,2,1,2,3,3,5,1,3,5,5,4,2,4,4,5,NaN
1,kumahimeinlove,3,NaN,2,3,5,4,3,2,3,3,2,3,3,3,4,NaN,NaN
2,NangmanLaws,5,4,4,2,2,3,2,3,2,4,4,4,3,3,3,5,NaN
3,iamnotlogical_,NaN,3,5,2,4,1,3,5,4,4,5,5,5,5,5,3,NaN
4,OuO76151129,NaN,3,2,0,0,3,0,3,4,5,1,2,0,3,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4743,juyonisaranghae,1,2,0,0,5,NaN,5,0,2,1,0,2,0,NaN,3,1,NaN
4744,KIM_Ryan10,2,3,4,2,3,NaN,0,1,4,3,3,4,4,4,2,4,NaN
4745,ssuing34,3,3,0,0,4,NaN,3,0,3,0,0,0,0,3,3,3,NaN
4746,CELE5TIA,4,3,1,0,4,3,5,0,1,5,5,5,0,3,4,5,NaN


In [22]:
#DB에 저장
from sqlalchemy import create_engine

db_connection_str = 'mysql+pymysql://'+'1zo'+':'+'multi1234!'+'@'+'13.112.232.65'+'/'+'tweets'
# 'mysql+pymysql://[db유저이름]:[db password]@[host address]/[db name]'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()
df_food.to_sql(name='food_db', con=db_connection, if_exists='replace',index=False)  

Exception during reset or similar
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/pymysql/connections.py", line 756, in _write_bytes
    self._sock.sendall(data)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py", line 2509, in do_rollback
    dbapi_connection.rollback()
  File "/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/pymysql/connections.py", line 479, in rollback
    self._execute_command(COMMAND.COM_

In [24]:
df_food.to_csv('./output/id_food.csv')